In [1]:
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
#from langchain.chains import RetrievalQA
from langchain.llms import Ollama  # ou OpenAI
from langchain_core.prompts import PromptTemplate
#from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [2]:
# Étape 1 : Charger les documents 
loader = PyPDFLoader("Introduction_to_Data_and_Data_Science.pdf")

In [3]:
pages = loader.load()

In [4]:
pages

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Analysis vs Analytics \nAlright! So… \nLet’s discuss the not-so-obvious differences \nbetween the terms analysis and analytics. \nDue to the similarity of the words, some people \nbelieve they share the same meaning, and thus \nuse them interchangeably. Technically, this \nisn’t correct. There is, in fact, a distinct \ndifference between the two. And the reason \nfor one often being used instead of the other \nis the lack of a transparent understanding \nof both. \nSo, let’s clear this up, shall we? \nFirst, we will start with analysis. \nConsider the following… \nYou have a huge dataset containing data of \nvarious types. Instead of tackli

In [5]:
pages[0]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Analysis vs Analytics \nAlright! So… \nLet’s discuss the not-so-obvious differences \nbetween the terms analysis and analytics. \nDue to the similarity of the words, some people \nbelieve they share the same meaning, and thus \nuse them interchangeably. Technically, this \nisn’t correct. There is, in fact, a distinct \ndifference between the two. And the reason \nfor one often being used instead of the other \nis the lack of a transparent understanding \nof both. \nSo, let’s clear this up, shall we? \nFirst, we will start with analysis. \nConsider the following… \nYou have a huge dataset containing data of \nvarious types. Instead of tacklin

In [6]:
len(pages)

6

In [7]:
# removing the character \n :
for i in range(len(pages)):
    pages[i].page_content = ' '.join(pages[i].page_content.split())

In [8]:
pages[0]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Analysis vs Analytics Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running

In [9]:
len(pages[0].page_content)

1541

In [10]:
# Etape 2 : Document splitting

#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(separator = ".", 
                                      chunk_size = 500, 
                                      chunk_overlap = 50)

In [11]:
doc = splitter.split_documents(pages)

In [12]:
doc

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Analysis vs Analytics Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both'),
 Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'modda

In [13]:
doc[0]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Analysis vs Analytics Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both')

In [14]:
len(doc[0].page_content)

444

In [15]:
len(doc)

22

In [16]:
# Étape 3 : Créer les embedding :
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\fares\AppData\Local\Temp\ipykernel_6840\2478700857.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [17]:
vector1 = embedding_model.embed_query(doc[3].page_content)

In [18]:
#vector1

In [19]:
len(vector1)

384

In [20]:
vector2 = embedding_model.embed_query(doc[5].page_content)

In [21]:
len(vector2)

384

In [22]:
vector3 = embedding_model.embed_query(doc[18].page_content)

In [23]:
len(vector3)

384

In [24]:
np.dot(vector1, vector2)

np.float64(0.43112179590131605)

In [25]:
np.dot(vector1, vector3)

np.float64(0.4829146500017961)

In [26]:
np.dot(vector2, vector3)

np.float64(0.31603760100297507)

In [27]:
np.linalg.norm(vector1), np.linalg.norm(vector2), np.linalg.norm(vector3)

(np.float64(0.9999999415411851),
 np.float64(1.0000000198679222),
 np.float64(1.000000015319548))

In [28]:
# Etape 4 : Creér la base vectorielle:

In [29]:
vectorstore = Chroma.from_documents(documents = doc, 
                                    embedding = embedding_model) 
                                    #persist_directory = "./intro-to-ds")

In [30]:
#vectorstore_from_directory = Chroma(persist_directory = "./intro-to-ds", 
#                                    embedding_function = embedding_model)

In [31]:
# Retriever:
retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k': 3, 
                                                      'lambda_mult': 0.7})

In [32]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024278600590>, search_type='mmr', search_kwargs={'k': 3, 'lambda_mult': 0.7})

In [33]:
#retrieved_docs = retriever.invoke(question)

In [34]:
#retrieved_docs

In [35]:
#for i in retrieved_docs:
#    print(f"Page Content: {i.page_content}\n----------\nLecture Title:{i.metadata['Lecture Title']}\n")

In [33]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [34]:
llm = Ollama(model="llama3")  # ou llm = OpenAI(api_key="...")

# ou on peut faire :
#qa_chain = RetrievalQA.from_chain_type(
#    llm=llm,
#    retriever=vectorstore.as_retriever(),
#    chain_type="stuff"
#)


C:\Users\fares\AppData\Local\Temp\ipykernel_6840\4226933304.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")  # ou llm = OpenAI(api_key="...")


In [35]:
question = "What software do data scientists use?"

In [36]:
chain = ({'context': retriever, 
         'question': RunnablePassthrough()} 
         | prompt_template 
         | llm 
         | StrOutputParser())

In [42]:
response = chain.invoke(question)

In [43]:
response

'Based on the provided context, data scientists use software such as Apache Hadoop, Apache Hbase, and Mongo DB for working with big data. They also employ programming languages and tools like SPSS for traditional data and statistical analysis.\n\nResources: Introduction to Data and Data Science'

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

app = FastAPI(title="DocBot")

# Vectorstore & Retriever
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
vectordb = Chroma(persist_directory="chroma_db", embedding_function=embeddings)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k":4})

# LLM
llm = OpenAI(temperature=0)
prompt_template = """Tu es un assistant documentaire. Réponds uniquement en utilisant les informations fournies.
Contexte:
{context}

Question: {question}

Réponds avec des sources si possible.
"""
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",
                                    retriever=retriever,
                                    return_source_documents=True,
                                    chain_type_kwargs={"prompt": PromptTemplate.from_template(prompt_template)})

class Query(BaseModel):
    question: str
    top_k: int = 4

@app.post("/query")
def query(q: Query):
    result = chain({"query": q.question})
    return {"answer": result["result"], "sources": [d.metadata for d in result["source_documents"]]}
